# Sample

In [2]:
import stock 
import plotly.graph_objs as go

import pandas as pd 
import vnstock 

df = stock.get_daily_OHLC(from_date='18/01/2022', to_date='23/01/2024', page_size=1000)

df['TradingDate'] = pd.to_datetime(df['TradingDate'], format='%d/%m/%Y').dt.strftime('%Y/%m/%d')
df


ModuleNotFoundError: No module named 'vnstock'

In [40]:
def candlestick_chart(df, title='Candlestick Chart with MA and Volume', x_label='Date', y_label='Price', ma_periods=None, show_volume=True, figure_size=(15, 8), reference_period=None, colors=('#00F4B0', '#FF3747'), reference_colors=('blue', 'black')):
    """
    Generate a candlestick chart with optional Moving Averages (MA) lines, volume data, and reference lines.

    Parameters:
    - df: DataFrame with candlestick data ('time', 'open', 'high', 'low', 'close', 'volume', 'ticker').
    - title: Title of the chart.
    - x_label: Label for the x-axis.
    - y_label: Label for the y-axis.
    - ma_periods: List of MA periods to calculate and plot (e.g., [10, 50, 200]).
    - show_volume: Boolean to indicate whether to display volume data.
    - figure_size: Tuple specifying the figure size (width, height).
    - reference_period: Number of days to consider for reference lines (e.g., 90).
    - colors: Tuple of color codes for up and down candles (e.g., ('#00F4B0', '#FF3747')).
    - reference_colors: Tuple of color codes for reference lines (e.g., ('black', 'blue')).

    Returns:
    - Plotly figure object.
    """
    # Create the base candlestick chart
    candlestick_trace = go.Candlestick(
        x=df['TradingDate'],
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='Candlestick',
    )

    # Create a figure
    fig = go.Figure(data=[candlestick_trace])

    # Add volume data if specified
    if show_volume:
        df['Volume']=df['Volume'].astype(float)
        volume_trace = go.Bar(
            x=df['TradingDate'],
            y=df['Volume'],
            name='Volume',
            yaxis='y2',  # Use the secondary y-axis for volume
            marker=dict(color=[colors[0] if close >= open else colors[1] for close, open in zip(df['Close'], df['Open'])]),  # Match volume color to candle color
        )

        fig.add_trace(volume_trace)

    # Add Moving Averages (MA) lines if specified
    if ma_periods:
        for period in ma_periods:
            ma_name = f'{period}-day MA'
            df[ma_name] = df['Close'].rolling(period).mean()

            ma_trace = go.Scatter(
                x=df['TradingDate'],
                y=df[ma_name],
                mode='lines',
                name=ma_name,
            )

            fig.add_trace(ma_trace)

    # Add straight reference lines for the highest high and lowest low
    if reference_period:
        df['lowest_low'] = df['Low'].rolling(reference_period).min()
        df['highest_high'] = df['High'].rolling(reference_period).max()

        lowest_low_trace = go.Scatter(
            x=df['TradingDate'],
            y=[df['lowest_low'].iloc[-1]] * len(df),  # Create a straight line for lowest low
            mode='lines',
            name=f'Lowest Low ({reference_period} days)',
            line=dict(color=reference_colors[0], dash='dot'),
        )

        highest_high_trace = go.Scatter(
            x=df['TradingDate'],
            y=[df['highest_high'].iloc[-1]] * len(df),  # Create a straight line for highest high
            mode='lines',
            name=f'Highest High ({reference_period} days)',
            line=dict(color=reference_colors[1], dash='dot'),
        )

        fig.add_trace(lowest_low_trace)
        fig.add_trace(highest_high_trace)

    # Customize the chart appearance
    fig.update_layout(
        title=title,
        xaxis_title=x_label,
        yaxis_title=y_label,
        xaxis_rangeslider_visible=True,
        yaxis2=dict(
            title='Volume',
            overlaying='y',
            side='right',
        ),
        width=figure_size[0] * 100,  # Convert short form to a larger size for better readability
        height=figure_size[1] * 100,
        margin=dict(l=50, r=50, t=70, b=50),  # Adjust margins for space between title and legend
    )

    return fig

In [50]:
candlestick_chart(df, ma_periods=[5], show_volume= True, reference_period=200)

In [1]:

import stock
stock.get_securities_list(page_size=1000
                          )

,Market,Symbol,StockName,StockEnName
0,HOSE,AAA,CTCP NHUA AN PHAT XANH,An Phat Bioplastics Joint Stock Company
1,HOSE,AAM,CTCP THUY SAN MEKONG,Mekong Fisheries Joint Stock Company
2,HOSE,AAT,CTCP TD TIEN SON TH,Tien Son Thanh Hoa Group Joint Stock Company
3,HOSE,ABR,CTCP DT NHAN HIEU VIET,Viet Brand Invest Joint Stock Company
4,HOSE,ABS,CTCP DV NN BINH THUAN,BinhThuan Agriculture Services Joint Stock Com...
...,...,...,...,...
616,HOSE,VSI,CTCP DT&XD CAP THOAT NUOC,Water Supply Sewerage Construction and Investm...
617,HOSE,VTB,CTCP VIETTRONICS TAN BINH,Viettronics Tan Binh Joint Stock Company
618,HOSE,VTO,CTCP VT XANG DAU VITACO,Vietnam Tanker Joint Stock Company
619,HOSE,YBM,CTCP KHOANGSAN CN YEN BAI,Yen Bai Industry Mineral Joint Stock Company


In [65]:
import requests as rq
data = rq.get(
        f'https://apipubaws.tcbs.com.vn/tcanalysis/v1/finance/AAA/financialratio?yearly=0&isAll=true').json()
print(data)

[{'ticker': 'AAA', 'quarter': 5, 'year': 2023, 'priceToEarning': 13.9, 'priceToBook': 0.8, 'valueBeforeEbitda': 9.8, 'dividend': None, 'roe': 0.054, 'roa': 0.026, 'daysReceivable': 64, 'daysInventory': 40, 'daysPayable': 22, 'ebitOnInterest': 1.7, 'earningPerShare': 760, 'bookValuePerShare': 13992, 'interestMargin': None, 'nonInterestOnToi': None, 'badDebtPercentage': None, 'provisionOnBadDebt': None, 'costOfFinancing': None, 'equityOnTotalAsset': 0.464, 'equityOnLoan': None, 'costToIncome': None, 'equityOnLiability': 1.1, 'currentPayment': 1.5, 'quickPayment': 1.3, 'epsChange': 0.906, 'ebitdaOnStock': 1686, 'grossProfitMargin': 0.088, 'operatingProfitMargin': 0.025, 'postTaxMargin': 0.023, 'debtOnEquity': 0.5, 'debtOnAsset': 0.3, 'debtOnEbitda': 4.7, 'shortOnLongDebt': 7.3, 'assetOnEquity': 2.2, 'capitalBalance': 1941, 'cashOnEquity': 0.411, 'cashOnCapitalize': 0.601, 'cashCirculation': 82, 'revenueOnWorkCapital': 5.7, 'capexOnFixedAsset': -0.297, 'revenueOnAsset': 1.1, 'postTaxOnPreT

In [66]:
import streamlit as st
import matplotlib.pyplot as plt
import requests

# Your data
data = requests.get(f'https://apipubaws.tcbs.com.vn/tcanalysis/v1/finance/AAA/financialratio?yearly=1&isAll=true')

# Extracting data for the chart
years = [entry['year'] for entry in data]
pe_ratios = [entry['priceToEarning'] for entry in data]
pb_ratios = [entry['priceToBook'] for entry in data]

# Plotting the data
fig, ax = plt.subplots()
ax.plot(years, pe_ratios, label='Price to Earning')
ax.plot(years, pb_ratios, label='Price to Book')

# Adding labels and title
ax.set_xlabel('Year')
ax.set_ylabel('Ratio')
ax.set_title('Price to Earning and Price to Book Over Years')
ax.legend()

# Streamlit app
st.title('Stock Ratios Over Years')
st.pyplot(fig)


TypeError: byte indices must be integers or slices, not str

In [10]:
import pandas as pd 
import requests
import stock
import pandas as pd 
import requests
import stock
import time

tickers = stock.get_securities_list(page_size=1000)['Symbol']
tickers
data = []
i = 0
for ticker in tickers:
    json = requests.get(
        f"https://apipubaws.tcbs.com.vn/tcanalysis/v1/ticker/{ticker}/overview").json()
    # df = pd.json_normalize(json)
    # display(df)
    data.append(json)
    i += 1
    print(i)
    
    
df = pd.json_normalize(data)
df.to_csv('stock.csv', index=False)





1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
